<a href="https://colab.research.google.com/github/mickias-zerfu/Build-Basic-Generative-Adversarial-Networks-GANs/blob/main/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
!git clone https://github.com/NVlabs/stylegan.git
!ls /content/stylegan/

Cloning into 'stylegan'...
remote: Enumerating objects: 86, done.
remote: Total 86 (delta 0), reused 0 (delta 0), pack-reused 86
Unpacking objects: 100% (86/86), done.
config.py	     LICENSE.txt	    run_metrics.py
dataset_tool.py      metrics		    stylegan-teaser.png
dnnlib		     pretrained_example.py  training
generate_figures.py  README.md		    train.py


In [ ]:
import sys
sys.path.insert(0, "/content/stylegan")
import dnnlib

In [ ]:
# pip install python-resize-image

In [ ]:
# resize all the images to same size
# import os
# from tqdm import tqdm
# import cv2
# from PIL import Image
# from resizeimage import resizeimage
# path = '/content/drive/MyDrive/samesize/samesized/'
# for filename in tqdm(os.listdir(path),desc ='reading images ...'):
#   image = Image.open(path+filename)
#   print(image)
#   rgba_image = image
#   rgba_image.load()
#   background = Image.new("RGB", rgba_image.size, (255, 255, 255))
#   background.paste(rgba_image, mask = rgba_image.split()[3])
#   image =background
#   # image = image.resize((512,512))
  # image.save(path+filename, "JPEG", quality=100)

In [ ]:
! python /content/stylegan/dataset_tool.py create_from_images /content/stylegan/datasets/custom-dataset /content/drive/MyDrive/samesized




Loading images from "/content/drive/MyDrive/samesized"
Creating dataset "/content/stylegan/datasets/custom-dataset"



/content/stylegan/dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 3474 images.


In [ ]:
! python /content/stylegan/train.py




Creating the run dir: results/00001-sgan-/content/stylegan/datasets/custom-dataset-1gpu
Copying files to the run dir
dnnlib: Running training.training_loop.training_loop() on localhost...



Streaming data using training.dataset.TFRecordDataset...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.
Dataset shape = [3, 512, 512]
Dynamic range = [0, 255]
Label size    = 0
Constructing networks...







G                             Params    OutputShape         

In [ ]:
# !zip -r /content/results.zip /content/results
!zip -r /content/gan.zip /content/stylegan

In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config
def main():
  # Initialize TensorFlow.
  tflib.init_tf()
  url = '/content/results/00000-sgan-/content/stylegan/datasets/custom-dataset-1gpu/network-snapshot-000140.pkl'
  with open(url,'rb') as f :
    _G, _D, Gs = pickle.load(f)
# _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
# _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
# Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

# Print network details.
    Gs.print_layers()

    # Pick latent vector.
    rnd = np.random.RandomState()
    latents = rnd.randn(1, Gs.input_shape[1])

    # Generate image.
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)
    # Save image.
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, f'/content/example1.png')
    PIL.Image.fromarray(images[0], 'RGB').save(png_filename)
    #if __name__ == "__main__":
main()
